In [1]:
from libs.alibaba.alibaba import Alibaba
from libs.alibaba.inquiry import Inquiry
from libs.json import JSON

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import WebDriverException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotVisibleException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains

from datetime import datetime
from bs4 import BeautifulSoup
from pyquery import PyQuery as pq

import json
import requests
import time
import re
import math

import types
import traceback
import threading

import logging
from logging.handlers import TimedRotatingFileHandler

In [2]:
market = JSON.deserialize('.', 'storage', 'markets.json')['Eyelashes']
market

. storage markets.json
.\storage\markets.json was deserialized!


{'name': 'Eyelashes',
 'directory': 'E:\\glitter\\Eyelashes',
 'lid': 'eyelash@glittergroupcn.com',
 'lpwd': 'Wangzhen68970747'}

In [3]:
inquiry = Inquiry(market)
inquiry.login()

primary 打开网址：https://i.alibaba.com
primary 等待登陆页面加载 ... ...
primary 输入登录信息， 并登录
success 成功 登录 阿里巴巴 国际站！用户名：Jeremy


In [4]:
def open_inquiry_in_new_tab(self, inquiry):
    href = self.browser.find_element_by_css_selector('a[data-trade-id="'+inquiry['id']+'"]').get_attribute('href')
    self.browser.execute_script("window.open('');")
    self.browser.switch_to.window(self.browser.window_handles[1])
    self.browser.get(href)

inquiry.open_inquiry_in_new_tab = types.MethodType(open_inquiry_in_new_tab, inquiry)

In [ ]:
def get_inquiries(self):
    html = pq(self.browser.page_source)
    inquiry_list = html.find('.ui2-list-body .aui2-grid-wraper')
    inquiries = []
    for elem in inquiry_list:
        div = pq(elem)
        item = {}
        item['url'] = div.find('a').attr('href')
        text = div.find('div.spec-inquiry-id').text().strip()
        item['id'] = re.sub('询价单号：', '', text)
        item['date'] = div.find('div.spec-inquiry-id+div').text().strip()
        item['tags'] = []
        for span in div.find('div.aui-tags span'):
            item['tags'].append(pq(span).text())
        item['is_replied'] = True if div.find('td:nth-child(2) i[title="已回复"]') else False
        item['title'] = div.find('td:nth-child(3)').text()
        item['buyer'] = div.find('td:nth-child(4)').text().strip()
        item['is_buyer_online'] = True if div.find('td:nth-child(5) div.atm-online') else False
        text = div.find('td:nth-child(6) i').attr('title')
        try:
            item['buyer_country'] = re.sub(' \[.*\]', '', text)
            item['buyer_local_time'] = re.search('\[Local Time: (.*)\]', text).group(1)
        except AttributeError:
            item['buyer_country'] = text
            item['buyer_local_time'] = None
        item['responsible_person'] = div.find('td:nth-child(7)').text()
        item['status'] = div.find('td:nth-child(8)').text()
        if '新询盘' in item['status']:
            item['is_new'] = True
        else:
            item['is_new'] = False
        inquiries.append(item)
    return inquiries

inquiry.get_inquiries = types.MethodType(get_inquiries, inquiry)

In [5]:
inquiry.load_url()
inquiries = inquiry.get_inquiries()

In [11]:
for enquiry in inquiries:
    if enquiry['tags'] or enquiry['is_replied']:
        continue
        
    inquiry.open_inquiry_in_new_tab(enquiry)

    conversation = inquiry.get_conversation()

    messages = conversation['messages']
    buyer = conversation['buyer']

    is_replied = False
    reply_count = 0
    for msg in messages:
        if msg['position'] == 'left':
            is_replied = False
        elif msg['position'] == 'right' and 'Seller Assistant' not in msg['user']:
            is_replied = True
            reply_count += 1

    if reply_count == 0 and is_replied is False:
        print('this is a new inquiry.', enquiry['id'])
        pass
    elif reply_count !=0 and is_replied is False:
        print('this inquiry is not replied yet!', enquiry['id'])
        pass
    else:
        print('this inquiry was replied.', enquiry['id'])
        pass

    inquiry.close_inquiry_and_switch_back()
    break

this inquiry was replied. 11947313067


{'messages': [{'position': 'left', 'user': 'prestl bernhard', 'date': '2018-07-05 21:10', 'content': 'hi,\ni want to know the price of the product 11A31, if i buy 1000 pairs. please give me best price you kan.\nbest regards\nprestl bernhard'}, {'position': 'right', 'user': 'Seller Assistant', 'date': '22:41', 'content': ''}], 'buyer': {'email': '隐藏', 'name': 'prestl bernhard'}}
this is a new inquiry.


In [ ]:
inquiry.close_inquiry_and_switch_back()

In [8]:
buyer

{'email': '隐藏', 'name': 'prestl bernhard'}

In [ ]:
messages

In [ ]:
inquiries[0]

In [7]:
enquiry

{'url': '/message/default.htm#feedback/detail/all/11947313067/11947313067/MC1IDX1IwKXUHujRqXIIRl2n8a6Mlpuf9Xc9B8yay0eTeGy1lewW7WMPLAcIwORTYH1cXSJ/',
 'id': '11947313067',
 'date': '21:10',
 'tags': [],
 'is_replied': False,
 'title': 'Product inquiry about Qingdao Factory Wholesale Own Brand Lashes 3D Private Label Mink Eyelash With Customized packaging from prestl bernhard.',
 'buyer': 'prestl bernhard',
 'is_buyer_online': False,
 'buyer_country': 'United States',
 'buyer_local_time': '07: 40 Thu Jul 05',
 'responsible_person': 'Helen Wang',
 'status': '新询盘\n已等待1时29分',
 'is_new': True}